<a href="https://colab.research.google.com/github/aysharega/M.Tech-Project-Thesis/blob/main/3D_CNN_in_3D_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn.preprocessing import OneHotEncoder
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import random

In [4]:
tf.config.run_functions_eagerly(True)

In [21]:
with h5py.File("/content/full_dataset_vectors.h5", "r") as hf:    
    # Split the data into training/test features/targets
    x_train = hf["X_train"][:]
    y_train = hf["y_train"][:]
    x_test = hf["X_test"][:] 
    y_test = hf["y_test"][:]

threshold, upper, lower = 0.2, 1, 0
X_train = np.where(X_train>threshold, upper, lower)
X_test = np.where(X_test>threshold, upper, lower)

x_train = x_train.reshape(x_train.shape[0], 1, 16,16,16)
x_test = x_test.reshape(x_test.shape[0], 1, 16,16,16)
onehot_encoder = OneHotEncoder(sparse=False)
y_train_onehot = onehot_encoder.fit_transform(y_train.reshape(len(y_train), 1))
y_test_onehot = onehot_encoder.transform(y_test.reshape(len(y_test), 1))

print("x_train", x_train.shape)
print("y_train", y_train.shape)
print("y_train_onehot", y_train_onehot.shape)

print("x_test", x_test.shape)
print("y_test", y_test.shape)
print("y_test_onehot", y_test_onehot.shape)

x_train (10000, 1, 16, 16, 16)
y_train (10000,)
y_train_onehot (10000, 10)
x_test (2000, 1, 16, 16, 16)
y_test (2000,)
y_test_onehot (2000, 10)


In [22]:
plot_idx = random.randint(0, x_train.shape[0])
plot_img_3d = np.squeeze(x_train[plot_idx])
plot_label = y_train[plot_idx]

plot_data = []
for x in range(0,16):
    for y in range(0,16):
        for z in range(0,16):
            val = plot_img_3d[x,y,z]
            plot_data.append([x,y,z,val])

plot_df = pd.DataFrame(plot_data, columns=["x", "y", "z", "val"])
plot_df = plot_df.loc[plot_df["val"] > 0]
fig = go.Figure(data=[go.Scatter3d(x=plot_df['x'], y=plot_df['y'], z=plot_df['z'],
                                   mode='markers',
                                   marker=dict(
                                   size=4,       
                                   colorscale='Viridis',
                                   opacity=0.8))])
fig.show()

In [23]:
model = Sequential([
    layers.Conv3D(8, (3,3,3), activation='relu', padding='same', input_shape=(1, 16,16,16)),
    layers.Conv3D(16, (3,3,3), activation='relu', padding='same'),
    layers.MaxPooling3D((2,2,2), padding='same'),
    
    layers.Conv3D(32, (3,3,3), activation='relu', padding='same'),
    layers.Conv3D(64, (3,3,3), activation='relu', padding='same'),
    layers.MaxPooling3D((2,2,2), padding='same'),
    
    layers.Conv3D(16, (3,3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.GlobalAveragePooling3D(),
    layers.Flatten(),
    
    layers.Dense(units=1024, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(units=256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(units=10, activation='softmax'),
])

In [24]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 1, 16, 16, 8)      3464      
                                                                 
 conv3d_1 (Conv3D)           (None, 1, 16, 16, 16)     3472      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 1, 8, 8, 16)      0         
 )                                                               
                                                                 
 conv3d_2 (Conv3D)           (None, 1, 8, 8, 32)       13856     
                                                                 
 conv3d_3 (Conv3D)           (None, 1, 8, 8, 64)       55360     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 1, 4, 4, 64)      0         
 3D)                                                    

In [25]:
model.fit(x=x_train, y=y_train_onehot, batch_size=128, epochs=50, validation_split=0.2)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning:

Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.



Epoch 1/50
63/63 [==============================] - 25s 391ms/step - loss: 1.9808 - acc: 0.2715 - val_loss: 2.2376 - val_acc: 0.1275
Epoch 2/50
63/63 [==============================] - 25s 391ms/step - loss: 1.4886 - acc: 0.4679 - val_loss: 2.1766 - val_acc: 0.2540
Epoch 3/50
63/63 [==============================] - 25s 390ms/step - loss: 1.2300 - acc: 0.5630 - val_loss: 1.8151 - val_acc: 0.3720
Epoch 4/50
63/63 [==============================] - 26s 410ms/step - loss: 1.1076 - acc: 0.6101 - val_loss: 1.5001 - val_acc: 0.5695
Epoch 5/50
63/63 [==============================] - 24s 389ms/step - loss: 1.0489 - acc: 0.6345 - val_loss: 1.3772 - val_acc: 0.5600
Epoch 6/50
63/63 [==============================] - 26s 409ms/step - loss: 0.9848 - acc: 0.6586 - val_loss: 1.2553 - val_acc: 0.5695
Epoch 7/50
63/63 [==============================] - 24s 388ms/step - loss: 0.9542 - acc: 0.6591 - val_loss: 1.1964 - val_acc: 0.5845
Epoch 8/50
63/63 [==============================] - 26s 406ms/step - 

In [26]:
model.evaluate(x=x_test, y=y_test_onehot)

 3/63 [>.............................] - ETA: 2s - loss: 1.3539 - acc: 0.7188

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning:

Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.



63/63 [==============================] - 2s 37ms/step - loss: 1.9073 - acc: 0.6670


[1.907344937324524, 0.6669999957084656]